In [ ]:
"""
Author: Hanshuo Wu
This file is for projection of 2D material segmentation results to 3D world.
"""

In [ ]:
import numpy as np
import pye57
import os

In [ ]:
from MatchPointPixel import PointCloud

In [ ]:
folder = "to/your/folder"  # Change to your local path
file = "your_file.e57"
file_path = os.path.join(folder, file)
e57 = pye57.E57(file_path)
PC=PointCloud(folder+file, RGB = False)   # A Point Cloud Object

### Create a point cloud object

Raw data read here are in camera coordinate!  

i.e., Raw data read (CartesianXYZ) + Translation Matrix = Coordinates in CloudCompare  

In [ ]:
header = e57.get_header(0)
print("Number of Points=", header.point_count)
print("Rotation Matrix:" , header.rotation_matrix)
print("Translation Matrix:" , header.translation)
# all the header information can be printed using:
# print(" --- The raw data is: ---")
# for line in header.pretty_print():
#     print(line)


### Segment 2D data  

#### Classification

1 - Rustication  
2 - Brick  
3 - Stucco  
4 - Wood  
5......

In [ ]:
from ultralytics import YOLO

import torch
import plotly.graph_objects as go
import cv2

In [ ]:
# Get list of 6 images
image=PC.image_extract(RGB = False)

In [ ]:
model = YOLO("weights/best.pt")  # Change the path

In [ ]:
### Get predict result
results=[]
for i in range(6):
    results.append(model.predict(image[i],conf=0.25))

In [ ]:
# Squeeze into a single mask for each result
single_masks = []
for rlt in results:
    if rlt[0].masks != None:
        cal = (rlt[0].masks.masks) * ((rlt[0].boxes.cls+1).reshape(rlt[0].masks.masks.shape[0],1,1))   # Every mask/label has an offset=1, to avoid [class 0]
        cal = (torch.max(cal,0).values).numpy()  # reduce to one mask
        cal = np.rint(cv2.resize(cal, (2048,2048)))  # upscale the mask to the original input size
        single_masks.append(cal.astype(int)) # This is the single mask we need.
    else:
        single_masks.append(np.zeros((2048,2048)).astype(int))  # zeros(2048,2048) for images with no class.
#sys.getsizeof(single_masks)

### Data Visualization

In [ ]:
### Downsample randomly and visualize the original data.

data = np.zeros((PC.world_system["pts_world"]["X"].shape[0], 7))
data[:, 0] = PC.world_system["pts_world"]["X"]
data[:, 1] = PC.world_system["pts_world"]["Y"]
data[:, 2] = PC.world_system["pts_world"]["Z"]
data[:, 3] = PC.world_system["pts_world"]["R"]
data[:, 4] = PC.world_system["pts_world"]["G"]
data[:, 5] = PC.world_system["pts_world"]["B"]
# Channel 6 is the classification

#data = np.asarray(cloud.points)

n_points = data.shape[0]
idxs_sampling = np.random.choice(data.shape[0], np.min([10*(10**4), data.shape[0]]))
data = data[idxs_sampling, :]
print(data.shape)

trace=dict(type="scatter3d",
x=data[:, 0],
y=data[:, 1],
z=data[:, 2],
mode="markers",
marker=dict(color=[f"rgb({data[i, 3]}, {data[i, 4]}, {data[i, 5]})" for i in range(data.shape[0])])
)

fig = go.Figure(layout={"width": 600, "height": 400})
fig.add_trace(trace)
fig.update_traces(marker_size = 1)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
color_code = np.array([[255,255,255],  # white - class 0
                       [250,0,0], # red - Rustication - class 1
                       [0,250,0], # green - Brick - class 2
                       [0,0,250], # blue - Stucco  - class 3
                       [250,250,0], # yellow - Wood  - class 4
                      ])

In [ ]:
### Retreive and match each point"s label from images

for count, i in enumerate(data):
    point = np.array([[i[0]],[i[1]],[i[2]],[1]],dtype = float)
    #print(point)
    match = PC.bridge_point_to_pixel(point)   # match result
    if match is None:
        continue
    output = single_masks[match[0]] # Use index of the image to find the correct mask
    u = match[1][0]
    v = match[1][1]
    material = output[v][u]  # no need to int() since the output has been in np.int32
    #print(color_code[material])
    #print(match[2])
    i[6] = material  # Assign label
    #print("\r Processing: {0} / {1}".format(count,data.shape[0]),end="")

#This block took 4926s for 10000 points

In [ ]:
### Visualize the final result

#data = np.load("scan3-0725.npy")

trace=dict(type="scatter3d",
x=data[:, 0],
y=data[:, 1],
z=data[:, 2],
mode="markers",
marker=dict(color=[f"rgb({color_code[int(data[i, 6])][0]}, {color_code[int(data[i, 6])][1]}, {color_code[int(data[i, 6])][2]})" for i in range(data.shape[0])])
)

fig = go.Figure(layout={"width": 600, "height": 400})
fig.add_trace(trace)
fig.update_traces(marker_size = 1)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# color_code = np.array([[255,255,255],  # white - class 0
#                        [250,0,0], # red - Rustication - class 1
#                        [0,250,0], # green - Brick - class 2
#                        [0,0,250], # blue - Stucco  - class 3
#                        [250,250,0], # yellow - Wood  - class 4
#                       ])

In [ ]:
### save result
np.save("scan6-0829.npy",data)

In [ ]:
e57.close()